In [36]:
import os
from getpass import getpass

from dotenv import load_dotenv



In [37]:
import pstuts_rag

In [38]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from dataclasses import dataclass
@dataclass
class ApplicationParameters:
    filename = "data/test.json"
    embedding_model = "text-embedding-3-small"
    n_context_docs = 2

params = ApplicationParameters()

In [40]:

load_dotenv()

def set_api_key_if_not_present(key_name, prompt_message=""):
    if len(prompt_message) == 0:
        prompt_message=key_name
    if key_name not in os.environ or not os.environ[key_name]:
        os.environ[key_name] = getpass.getpass(prompt_message)

set_api_key_if_not_present("OPENAI_API_KEY")

# Data Preparation

First, we will read in the transcripts of the videos and convert them to Documents
with appropriate metadata.

In [53]:
from ast import Dict
import json

from pstuts_rag.loader import load_json_files
filename = ["../data/test.json","../data/dev.json"]
from typing import List, Dict, Any
data:List[Dict[str,Any]] = await load_json_files(filename)


In [56]:
[ d["title"] for d in data ]

['Get organized with layer groups',
 'Remove unwanted objects from photos',
 'Include vector graphics',
 'Remove unwanted content',
 'Add a central element',
 'Set the resolution',
 'Understand layers',
 'Adjust brightness and contrast',
 'Remove a large object',
 'Add text',
 'Replace a background using a layer mask']

## R - retrieval

Let's hit it with a semantic chunker.

In [43]:
from pstuts_rag.datastore import DatastoreManager
from qdrant_client import QdrantClient

client = QdrantClient(":memory:")

retriever_factory = DatastoreManager(qdrant_client=client,name="local_test")
if retriever_factory.count_docs() == 0:
    await retriever_factory.populate_database(raw_docs=data)

## A - Augmentation

We need to populate a prompt for LLM.


## Generation

We will use a 4.1-nano to generate answers.

In [44]:
from pstuts_rag.rag import RAGChainFactory

rag_factory = RAGChainFactory(retriever=retriever_factory.get_retriever())

In [45]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini",temperature=0)

In [46]:
get_videos = rag_factory.get_rag_chain(llm)
   


In [47]:
val = await get_videos.ainvoke({"question":"What are layers"})

In [48]:
val.additional_kwargs

{'refusal': None,
 'context': [Document(metadata={'video_id': 19172, 'title': 'Understand layers', 'desc': 'Learn what layers are and why they are so useful.', 'length': '00:04:44.75', 'group': 'test.json', 'source': 'https://images-tv.adobe.com/avp/vr/b758b4c4-2a74-41f4-8e67-e2f2eab83c6a/f810fc5b-2b04-4e23-8fa4-5c532e7de6f8/e268fe4d-e5c7-415c-9f5c-d34d024b14d8_20170727011753.1280x720at2400_h264.mp4', 'speech_start_stop_times': [[0.47, 3.41], [3.81, 9.13], [9.309999, 15.01], [15.299999, 20.57], [20.88, 23.3], [23.83, 27.93], [29.38, 32.79], [32.96, 33.92], [34.43, 40.21], [41.91, 45.37], [45.88, 49.01], [49.54, 55.130001], [55.72, 58.49], [58.72, 62.14]], 'start': 0.47, 'stop': 62.14, '_id': 21, '_collection_name': 'local_test'}, page_content="Layers are the building blocks of any image in Photoshop CC. So, it's important to understand, what layers are and why to use them - which we'll cover in this video. If you're following along, open this layered image from the downloadable practic

In [49]:
val.pretty_print()

================================== Ai Message ==================================

Layers are the building blocks of any image in Photoshop CC. You can think of layers like separate flat panes of glass stacked on top of each other. Each layer contains separate pieces of content. Some parts of a layer can be transparent, allowing you to see through to the layers below. This setup lets you edit parts of an image independently without affecting the rest of the image. You manage and work with layers in the Layers panel, where you can toggle their visibility on and off using the Eye icon. (See explanation around 0:28–1:00 and 1:25–2:32) 🎨🖼️
**References**:
[
  {
    "title": "Understand layers",
    "source": "https://images-tv.adobe.com/avp/vr/b758b4c4-2a74-41f4-8e67-e2f2eab83c6a/f810fc5b-2b04-4e23-8fa4-5c532e7de6f8/e268fe4d-e5c7-415c-9f5c-d34d024b14d8_20170727011753.1280x720at2400_h264.mp4",
    "start": 0.47,
    "stop": 62.14
  },
  {
    "title": "Understand layers",
    "source": "http

In [50]:
val.content

'Layers are the building blocks of any image in Photoshop CC. You can think of layers like separate flat panes of glass stacked on top of each other. Each layer contains separate pieces of content. Some parts of a layer can be transparent, allowing you to see through to the layers below. This setup lets you edit parts of an image independently without affecting the rest of the image. You manage and work with layers in the Layers panel, where you can toggle their visibility on and off using the Eye icon. (See explanation around 0:28–1:00 and 1:25–2:32) 🎨🖼️\n**References**:\n[\n  {\n    "title": "Understand layers",\n    "source": "https://images-tv.adobe.com/avp/vr/b758b4c4-2a74-41f4-8e67-e2f2eab83c6a/f810fc5b-2b04-4e23-8fa4-5c532e7de6f8/e268fe4d-e5c7-415c-9f5c-d34d024b14d8_20170727011753.1280x720at2400_h264.mp4",\n    "start": 0.47,\n    "stop": 62.14\n  },\n  {\n    "title": "Understand layers",\n    "source": "https://images-tv.adobe.com/avp/vr/b758b4c4-2a74-41f4-8e67-e2f2eab83c6a/f8